# bootstrap_words_02_add_frequency_embedding

- read in CSV -> spark df
- scrape word frequencies from API -> joblib dict
- scrape word embeddings from BERT -> joblib dict
- load frequency and embeddings, add to spark df
- NO date added column, just use version
- save as Delta table bronze.words


In [ ]:
%run "./00_setup.ipynb"

In [ ]:
# TODO: Parameterize as pipeline or notebook vars
_TARGET_DB_NAME = "bronze"
_TARGET_TABLE_NAME = "words"

In [ ]:
import os
import joblib
from pyspark.sql import functions as F
from pyspark.sql.types import *
from src.bronzeutils import words_schema
from src.constants import WORD_DATA_FILENAME, WORDLIST_PATH, WORDLIST_TEMP_CSV_FILENAME
from src.embeddingutils import get_word_embeddings, get_word_embeddings_threaded
from src.fileutils import get_local_path
from src.ngramsutils import get_word_frequencies_threaded
from src.sparkdbutils import create_db, create_unpartitioned_table

In [ ]:
# === FUNCTION DEFINITIONS ====

In [ ]:
def collect_word_data_with_checkpoint(words_list: list, 
                                     output_path: str,
                                     batch_size: int = 100,
                                     resume_job: bool = False):
    """
    Collect frequency and embedding data for a list of words with checkpointing.
    Saves final results as a joblib file that can be reused for multiple purposes.
    
    Args:
        words_list: List of words to process
        output_path: Path to save final joblib file with all data
        batch_size: Batch size for API calls (default 100 for API limits)
        resume_job: Whether to resume from existing checkpoint
    
    Returns:
        dict: Contains 'freq_dict' and 'embeddings_dict'
    """
    checkpoint_path = output_path.replace('.joblib', '_checkpoint.joblib')
    
    # Check for existing final output
    if os.path.exists(output_path) and not resume_job:
        print(f"Final output already exists at {output_path}")
        with open(output_path, 'rb') as f:
            return joblib.load(f)
    
    # Initialize or load checkpoint
    freq_dict = {}
    embeddings_dict = {}
    start_batch = 0
    
    if resume_job and os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'rb') as f:
            checkpoint_data = joblib.load(f)
            freq_dict = checkpoint_data.get('freq_dict', {})
            embeddings_dict = checkpoint_data.get('embeddings_dict', {})
            start_batch = checkpoint_data.get('last_batch', 0) + 1
        print(f"Resuming from batch {start_batch} with {len(freq_dict)} words already processed")
    
    total_batches = (len(words_list) + batch_size - 1) // batch_size
    
    try:
        # Process remaining batches
        for batch_idx in range(start_batch, total_batches):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, len(words_list))
            batch_words = words_list[start_idx:end_idx]
            
            print(f"Processing batch {batch_idx + 1}/{total_batches}: words {start_idx}-{end_idx-1}")
            
            # Get frequencies for this batch
            batch_freq_dict = get_word_frequencies_threaded(batch_words, max_workers=10)
            freq_dict.update(batch_freq_dict)
            
            # Get embeddings for this batch
            batch_embeddings_dict = get_word_embeddings_threaded(batch_words)
            embeddings_dict.update(batch_embeddings_dict)
            
            # Save checkpoint after each batch
            # Save embeddings as pure python lists
            checkpoint_data = {
                'freq_dict': freq_dict,
                'embeddings_dict': {k: v.tolist() for k, v in embeddings_dict.items()},
                'last_batch': batch_idx,
                'total_batches': total_batches
            }
            with open(checkpoint_path, 'wb') as f:
                joblib.dump(checkpoint_data, f)
            
            print(f"✅ Completed batch {batch_idx + 1}/{total_batches}\n")
    
    except Exception as err:
        print(f"Exception occurred! Resume job from batch {batch_idx}")
        print(f"Current progress: {len(freq_dict)} words processed")
        raise err
    
    # Save final output
    # Save embeddings not as numpy
    final_data = {
        'freq_dict': freq_dict,
        'embeddings_dict': {k: v.tolist() for k, v in embeddings_dict.items()},
        'processed_words': list(freq_dict.keys()),
        'total_words': len(freq_dict),
        'metadata': {
            'batch_size': batch_size,
            'total_batches': total_batches
        }
    }
    
    with open(output_path, 'wb') as f:
        joblib.dump(final_data, f)
    
    print(f"✅ Processing complete! Saved {len(freq_dict)} words to {output_path}")
    
    # Clean up checkpoint
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
        print("✅ Cleaned up checkpoint file")
    
    return final_data

In [ ]:
def apply_word_data_to_spark_df(df, word_data_path: str):
    """
    Apply frequency and embedding data to a Spark DataFrame using broadcast variables.
    
    Args:
        df: Spark DataFrame with a 'word' column
        word_data_path: Path to joblib file created by collect_word_data_with_checkpoint
    
    Returns:
        Spark DataFrame with 'frequency' and 'embedding' columns added
    """
    # Load the word data
    with open(word_data_path, 'rb') as f:
        word_data = joblib.load(f)
    
    freq_dict = word_data['freq_dict']
    embeddings_dict = word_data['embeddings_dict']
    
    print(f"Loaded data for {len(freq_dict)} words from {word_data_path}")
    
    # Create broadcast variables for efficient lookups
    freq_broadcast = spark.sparkContext.broadcast(freq_dict)
    embeddings_broadcast = spark.sparkContext.broadcast(embeddings_dict)
    
    frequency_udf = F.udf(lambda word: freq_broadcast.value.get(word, None), FloatType())
    embedding_udf = F.udf(lambda word: embeddings_broadcast.value.get(word, [None] * 768), ArrayType(FloatType()))
    
    # Add the new columns
    result_df = df.withColumn("frequency", frequency_udf(F.col("word"))) \
                 .withColumn("embedding", embedding_udf(F.col("word")))
    
    return result_df

In [ ]:
def validate_enriched_df(enriched_df):
    null_elements_df = enriched_df.filter(F.expr("exists(embedding, x -> x IS NULL)"))
    count_null_elements = null_elements_df.count()

    if count_null_elements > 0:
        raise Exception("Source data has null values in its embeddings")
    else:
        print("✅ No null values in any embeddings")

In [ ]:
def apply_schema(df, new_schema):
    current_schema = {field.name: field.dataType for field in df.schema.fields}
    exprs = []
    for field in new_schema.fields:
        if field.name in current_schema:
            current_type = current_schema[field.name]
            target_type = field.dataType

            if current_type == target_type:
                exprs.append(F.col(field.name).alias(field.name))
            else:
                exprs.append(F.col(field.name).cast(target_type).alias(field.name))
        else:
            exprs.append(F.lit(None).cast(field.dataType).alias(field.name))
    
    return df.select(*exprs)

In [ ]:
# === EXECUTION =====

In [ ]:
# Read CSV directly into Spark and collect word data
initial_schema = StructType([
    StructField("word", StringType(), True),
    StructField("letter_set", StringType(), True),
    StructField("version", IntegerType(), True)
])

temp_path = get_local_path(f"{WORDLIST_PATH}/{WORDLIST_TEMP_CSV_FILENAME}")
spark_df = spark.read.csv(temp_path, header=True, schema=initial_schema)

print(f"Read in {spark_df.count()} words from csv.")

In [ ]:
# Get list of words from the df
words_list = [row.word for row in spark_df.select("word").collect()]
word_data_path = get_local_path(f"{WORDLIST_PATH}/{WORD_DATA_FILENAME}")

In [ ]:
# Uncomment for testing only
words_list = words_list[617:113583:385]
spark_df = spark_df.filter(F.col("word").isin(words_list))
print(len(words_list), spark_df.count())

In [ ]:
# Collect word data (frequency from ngrams API, embeddings from local pre-trained BERT model)
# First run - set resume_job=False
word_data = collect_word_data_with_checkpoint(
    words_list, 
    word_data_path,
    batch_size=100,
    resume_job=False
)

# To resume if it crashes - set resume_job=True
# word_data = collect_word_data_with_checkpoint(
#     words_list, 
#     word_data_path,
#     batch_size=100,
#     resume_job=True
# )

In [ ]:
# Apply word data to DataFrame
enriched_df = apply_word_data_to_spark_df(spark_df, word_data_path)

In [ ]:
validate_enriched_df(enriched_df)

In [ ]:
enriched_df.filter(F.col("frequency").isNotNull()).show()

In [ ]:
final_df = apply_schema(enriched_df, words_schema)

In [ ]:
# Create the table
if not spark.catalog.databaseExists(_TARGET_DB_NAME):
    create_db(spark, _TARGET_DB_NAME)

create_unpartitioned_table(spark, final_df, _TARGET_TABLE_NAME, _TARGET_DB_NAME)

print(f"✅ Successfully created bootstrapped words table {_TARGET_DB_NAME}.{_TARGET_TABLE_NAME}")

In [ ]:
df2 = spark.sql(f"SELECT * FROM {_TARGET_DB_NAME}.{_TARGET_TABLE_NAME}")

In [ ]:
df2.filter(F.col("word").isin(words_list)).show(386)